# Foto Datos DP

In [1]:
from configparser import ConfigParser

from snowflake.connector import connect #type: ignore

config = ConfigParser()
config.read('config.ini')

conn = connect(
    user = config.get('SNOWFLAKE', 'USER'),
    password = config.get('SNOWFLAKE', 'PASSWORD'),
    account = config.get('SNOWFLAKE', 'ACCOUNT'),
    database = config.get('SNOWFLAKE', 'DATABASE'),
    warehouse = config.get('SNOWFLAKE', 'WAREHOUSE'),
    schema = config.get('SNOWFLAKE', 'SCHEMA'),
    role = config.get('SNOWFLAKE', 'ROLE'),
)

SEMANA_ACTUAL = 36
MES_ACTUAL = 9
ANIO_ACTUAL = 2023

SEMANA_PREVIA = 35
ANIO_SEMANA_PREVIA = 2023

MES_PREVIO = 8
ANIO_MES_PREVIO = 2023

ANIO_PREVIO = 2022

## Descargas App

In [2]:
instalaciones = conn.cursor().execute(f"""
    WITH

    ALL_INSTALLS AS (
        SELECT to_date(TIMESTAMP) AS FECHA FROM SEGMENT_EVENTS.GOLO_ANDROID_PROD.APPLICATION_INSTALLED
        UNION ALL
        SELECT to_date(TIMESTAMP) AS FECHA FROM SEGMENT_EVENTS.GOLO_IOS_PROD.APPLICATION_INSTALLED
        UNION ALL
        SELECT to_date(TIMESTAMP) AS FECHA FROM SEGMENT_EVENTS.DOMINOS_ANDROID_APP_PRODUCCION.APPLICATION_INSTALLED
        UNION ALL
        SELECT to_date(TIMESTAMP) AS FECHA FROM SEGMENT_EVENTS.DOMINOS_APP_PRODUCCION.APPLICATION_INSTALLED
    ),

    SEMANA_ACTUAL AS (
        SELECT
            count(*) AS SEMANA_ACTUAL
        FROM
            ALL_INSTALLS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
        ORDER BY
            FECHA DESC
    ),

    SEMANA_PREVIA AS (
        SELECT
            count(*) AS SEMANA_PREVIA
        FROM
            ALL_INSTALLS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_SEMANA_PREVIA}
        AND
            SEM_ALSEA = {SEMANA_PREVIA}
        ORDER BY
            FECHA DESC
    ),

    SEMANAL AS (
        SELECT
            *,
            (SEMANA_ACTUAL - SEMANA_PREVIA) / SEMANA_PREVIA AS PERCENT_V_SEMANA_PREVIA
        FROM
            SEMANA_ACTUAL
        JOIN
            SEMANA_PREVIA
    ),

    MES_ACTUAL AS (
        SELECT
            count(*) AS MES_ACTUAL
        FROM
            ALL_INSTALLS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            MES_ALSEA = {MES_ACTUAL}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
        ORDER BY
            FECHA DESC
    ),

    MES_PREVIO AS (
        SELECT
            count(*) AS MES_PREVIO
        FROM
            ALL_INSTALLS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_MES_PREVIO}
        AND
            MES_ALSEA = {MES_PREVIO}
        ORDER BY
            FECHA DESC
    ),

    MENSUAL AS (
        SELECT
            *,
            (MES_ACTUAL - MES_PREVIO) / MES_PREVIO AS PERCENT_V_MES_PREVIO
        FROM
            MES_ACTUAL
        JOIN
            MES_PREVIO
    ),

    ANIO_ACTUAL AS (
        SELECT
            count(*) AS ANIO_ACTUAL
        FROM
            ALL_INSTALLS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
        ORDER BY
            FECHA DESC
    ),

    ANIO_PREVIO AS (
        SELECT
            count(*) AS ANIO_PREVIO
        FROM
            ALL_INSTALLS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_PREVIO}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
        ORDER BY
            FECHA DESC
    ),

    ANUAL AS (
        SELECT
            *,
            (ANIO_ACTUAL - ANIO_PREVIO) / ANIO_PREVIO AS PERCENT_V_ANIO_PREVIO
        FROM
            ANIO_ACTUAL
        JOIN
            ANIO_PREVIO
    )

    SELECT
        *
    FROM
        SEMANAL
    JOIN
        MENSUAL
    JOIN
        ANUAL
    ;
""").fetch_pandas_all()

instalaciones.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO
0,54556,63556,-0.141607,118112,110215,0.071651,1248844,2561834,-0.512520


## Primeras compras

In [3]:
primeras_compras = conn.cursor().execute(f"""
    WITH

    TRANSACCIONES_OLO AS (
        SELECT DISTINCT
            lower(MXP.EMAIL) AS EMAIL,
            to_date(OLO.ORDER_DATE) AS FECHA
        FROM
        "SEGMENT_EVENTS"."DOMINOS_OLO"."MXPOWERSALESLOG" MXP
        INNER JOIN 
            "SEGMENT_EVENTS"."DOMINOS_OLO"."DPMSALES_FULL" OLO
        ON 
            OLO.ORDER_NUMBER = MXP.ORDERNUMBER AND OLO.LOCATION_CODE = MXP.STORENUMBER 
        AND 
            TO_CHAR(OLO.ORDER_DATE,'YYYY-MM-DD') = MXP.ORDERDATE
        WHERE 
            OLO.ORDER_STATUS_CODE = 4
        AND 
            OLO.LOCATION_CODE NOT IN ('13001' , '13006', '13021', '11000')
        AND 
            UPPER(OLO.SOURCE_CODE) IN ('ANDROID' , 'DESKTOP', 'IOS', 'MOBILE', 'WEB', 'ANDROID2', 'DESKTOP2', 'IOSAPP', 'MOBILE2', 'WHATSAPP')
    ),

    TRANSACCIONES_CLOUD AS (
        SELECT DISTINCT 
            lower(A.EMAIL) AS EMAIL,
            to_date(SUBSTRING(A.STOREORDERID,1,10)) AS FECHA
        FROM 
            "SEGMENT_EVENTS"."DOMINOS_GOLO"."VENTA_CLOUD" A
        WHERE 
            A.STOREID NOT LIKE '9%'
        AND 
            A.SOURCEORGANIZATIONURI IN ('order.dominos.com','resp-order.dominos.com','iphone.dominos.mx','android.dominos.mx') 
        AND 
            A.SOURCEORGANIZATIONURI IS NOT NULL
    ),

    TRANSACCIONES_TOTAL AS (
        SELECT * FROM TRANSACCIONES_CLOUD
        UNION ALL
        SELECT * FROM TRANSACCIONES_OLO
    ),

    PRIMERAS_COMPRAS_BY_USER AS (
        SELECT
            EMAIL,
            min(FECHA) AS FECHA
        FROM
            TRANSACCIONES_TOTAL
        GROUP BY
            EMAIL
    ),

    SEMANA_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS SEMANA_ACTUAL
        FROM
            PRIMERAS_COMPRAS_BY_USER
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
    ),

    SEMANA_PREVIA AS (
        SELECT
            count(DISTINCT EMAIL) AS SEMANA_PREVIA
        FROM
            PRIMERAS_COMPRAS_BY_USER
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_SEMANA_PREVIA}
        AND
            SEM_ALSEA = {SEMANA_PREVIA}
    ),

    SEMANAL AS (
        SELECT
            *,
            (SEMANA_ACTUAL - SEMANA_PREVIA) / SEMANA_PREVIA AS PERCENT_V_SEMANA_PREVIA
        FROM
            SEMANA_ACTUAL
        JOIN
            SEMANA_PREVIA
    ),

    MES_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS MES_ACTUAL
        FROM
            PRIMERAS_COMPRAS_BY_USER
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            MES_ALSEA = {MES_ACTUAL}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ),

    MES_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS MES_PREVIO
        FROM
            PRIMERAS_COMPRAS_BY_USER
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_MES_PREVIO}
        AND
            MES_ALSEA = {MES_PREVIO}
    ),

    MENSUAL AS (
        SELECT
            *,
            (MES_ACTUAL - MES_PREVIO) / MES_PREVIO AS PERCENT_V_MES_PREVIO
        FROM
            MES_ACTUAL
        JOIN
            MES_PREVIO
    ),

    ANIO_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS ANIO_ACTUAL
        FROM
            PRIMERAS_COMPRAS_BY_USER
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ),

    ANIO_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS ANIO_PREVIO
        FROM
            PRIMERAS_COMPRAS_BY_USER
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
        WHERE
            ANIO_ALSEA = {ANIO_PREVIO}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ),

    ANUAL AS (
        SELECT
            *,
            (ANIO_ACTUAL - ANIO_PREVIO) / ANIO_PREVIO AS PERCENT_V_ANIO_PREVIO
        FROM
            ANIO_ACTUAL
        JOIN
            ANIO_PREVIO
    )

    SELECT
        *
    FROM
        SEMANAL
    JOIN
        MENSUAL
    JOIN
        ANUAL
    ;
""").fetch_pandas_all()

primeras_compras.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO
0,58598,48274,0.213863,106872,100487,0.063541,808928,943176,-0.142336


## Registros Totales

In [4]:
primeras_compras = conn.cursor().execute(f"""
    WITH

    SEMANA_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS SEMANA_ACTUAL
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
    ),

    SEMANA_PREVIA AS (
        SELECT
            count(DISTINCT EMAIL) AS SEMANA_PREVIA
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        WHERE
            ANIO_ALSEA = {ANIO_SEMANA_PREVIA}
        AND
            SEM_ALSEA = {SEMANA_PREVIA}
    ),

    SEMANAL AS (
        SELECT
            *,
            (SEMANA_ACTUAL - SEMANA_PREVIA) / SEMANA_PREVIA AS PERCENT_V_SEMANA_PREVIA
        FROM
            SEMANA_ACTUAL
        JOIN
            SEMANA_PREVIA
    ),

    MES_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS MES_ACTUAL
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            MES_ALSEA = {MES_ACTUAL}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ),

    MES_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS MES_PREVIO
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        WHERE
            ANIO_ALSEA = {ANIO_MES_PREVIO}
        AND
            MES_ALSEA = {MES_PREVIO}
    ),

    MENSUAL AS (
        SELECT
            *,
            (MES_ACTUAL - MES_PREVIO) / MES_PREVIO AS PERCENT_V_MES_PREVIO
        FROM
            MES_ACTUAL
        JOIN
            MES_PREVIO
    ),

    ANIO_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS ANIO_ACTUAL
        FROM
           SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ),

    ANIO_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS ANIO_PREVIO
        FROM
           SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS 
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        WHERE
            ANIO_ALSEA = {ANIO_PREVIO}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ),

    ANUAL AS (
        SELECT
            *,
            CASE
                WHEN ANIO_PREVIO <> 0 THEN (ANIO_ACTUAL - ANIO_PREVIO) / ANIO_PREVIO
                ELSE 0
            END AS PERCENT_V_ANIO_PREVIO
        FROM
            ANIO_ACTUAL
        JOIN
            ANIO_PREVIO
    )

    SELECT
        *
    FROM
        SEMANAL
    JOIN
        MENSUAL
    JOIN
        ANUAL
    ;
""").fetch_pandas_all()

primeras_compras.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO
0,28782,35351,-0.185822,64056,81474,-0.213786,178636,0,0.000000


## Registros con compra

## Activos

In [5]:
activos = conn.cursor().execute(f"""
    WITH

    PERIODO_ACTUAL AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
    ),

    SEMANA_PREVIA AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_PREVIA}
    ),

    MES_PREVIO AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            MES_ALSEA = {MES_PREVIO}
    ),

    ANIO_PREVIO AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_PREVIO}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
    ),

    TRANSACCIONES_OLO AS (
        SELECT DISTINCT
            lower(MXP.EMAIL) AS EMAIL,
            ORDER_DATE AS FECHA,
            TO_CHAR(ORDER_DATE,'YYYY-MM-DD')||LOCATION_CODE||OLO.ORDER_NUMBER AS ORDER_ID,
            OLO.ORDERFINALPRICE / 1.16 AS ORDER_AMOUNT
        FROM
        "SEGMENT_EVENTS"."DOMINOS_OLO"."MXPOWERSALESLOG" MXP
        INNER JOIN 
            "SEGMENT_EVENTS"."DOMINOS_OLO"."DPMSALES_FULL" OLO
        ON 
            OLO.ORDER_NUMBER = MXP.ORDERNUMBER AND OLO.LOCATION_CODE = MXP.STORENUMBER 
        AND 
            TO_CHAR(OLO.ORDER_DATE,'YYYY-MM-DD') = MXP.ORDERDATE
        WHERE 
            OLO.ORDER_STATUS_CODE = 4
        AND 
            OLO.LOCATION_CODE NOT IN ('13001' , '13006', '13021', '11000')
        AND 
            UPPER(OLO.SOURCE_CODE) IN (
                'ANDROID', 
                'DESKTOP', 
                'IOS', 
                'MOBILE', 
                'WEB', 
                'ANDROID2', 
                'DESKTOP2', 
                'IOSAPP', 
                'MOBILE2', 
                'WHATSAPP'
            )
    ),

    TRANSACCIONES_CLOUD AS (
        SELECT DISTINCT 
            lower(A.EMAIL) AS EMAIL,
            to_date(SUBSTRING(A.STOREORDERID,1,10)) AS FECHA,
            to_char(FECHA) || A.STOREID ||A.StoreOrderID AS ORDER_ID,
            PAYMENTSAMOUNT / 1.16 AS ORDER_AMOUNT
        FROM 
            "SEGMENT_EVENTS"."DOMINOS_GOLO"."VENTA_CLOUD" A
        WHERE 
            A.STOREID NOT LIKE '9%'
        AND 
            A.SOURCEORGANIZATIONURI IN (
                'order.dominos.com', 
                'resp-order.dominos.com', 
                'iphone.dominos.mx', 
                'android.dominos.mx'
            ) 
        AND 
            A.SOURCEORGANIZATIONURI IS NOT NULL
    ),

    TRANSACCIONES_OLO_Y_CLOUD AS (
        SELECT * FROM TRANSACCIONES_OLO
        UNION ALL
        SELECT * FROM TRANSACCIONES_CLOUD
    ),

    ACTIVOS_PERIODO_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS CLIENTES
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            PERIODO_ACTUAL
        ON
            FECHA BETWEEN FECHA_INICIO AND FECHA_FIN
    ),

    ACTIVOS_SEMANA_PREVIA AS (
        SELECT
            count(DISTINCT EMAIL) AS CLIENTES
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            SEMANA_PREVIA
        ON
            FECHA BETWEEN FECHA_INICIO AND FECHA_FIN
    ),

    ACTIVOS_MES_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS CLIENTES
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            MES_PREVIO
        ON
            FECHA BETWEEN FECHA_INICIO AND FECHA_FIN
    ),

    ACTIVOS_ANIO_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS CLIENTES
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            ANIO_PREVIO
        ON
            FECHA BETWEEN FECHA_INICIO AND FECHA_FIN
    )

    SELECT
        ACTIVOS_PERIODO_ACTUAL.CLIENTES AS SEMANA_ACTUAL,
        ACTIVOS_SEMANA_PREVIA.CLIENTES AS SEMANA_PREVIA,
        SEMANA_ACTUAL / SEMANA_PREVIA AS PERCENT_V_SEMANA_PREVIA,

        ACTIVOS_PERIODO_ACTUAL.CLIENTES AS MES_ACTUAL,
        ACTIVOS_MES_PREVIO.CLIENTES AS MES_PREVIO,
        MES_ACTUAL / MES_PREVIO AS PERCENT_V_MES_PREVIO,

        ACTIVOS_PERIODO_ACTUAL.CLIENTES AS ANIO_ACTUAL,
        ACTIVOS_ANIO_PREVIO.CLIENTES AS ANIO_PREVIO,
        ANIO_ACTUAL / ANIO_PREVIO AS PERCENT_V_ANIO_PREVIO
    FROM
        ACTIVOS_PERIODO_ACTUAL
    JOIN
        ACTIVOS_SEMANA_PREVIA
    JOIN  
        ACTIVOS_MES_PREVIO
    JOIN
        ACTIVOS_ANIO_PREVIO
    ;
""").fetch_pandas_all()

activos.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO
0,1918253,1860546,1.031016,1918253,1818831,1.054663,1918253,1982923,0.967387
